# Introduction

QA plots for the Silicon Seeding

In [1]:
# imports to write dynamic markdown contents
import os
from IPython.display import display, Markdown, Latex
from IPython.display import HTML

In [2]:
# turn off/on code for the result HTML page

display(Markdown('*For the result HTML page:* '))
    
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 $('div.jp-CodeMirrorEditor').hide(); // for newer versions of nbviewer.org
 } else {
 $('div.input').show();
 $('div.jp-CodeMirrorEditor').show(); // for refer versions of nbviewer.org
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

*For the result HTML page:* 

In [3]:
import os.path

# readme file of the macros, available if run under JenkinsCI
# https://github.com/sPHENIX-Collaboration/utilities/blob/master/jenkins/built-test/test-tracking-qa.sh 

macro_markdown = 'Fun4All-macros-README.md'

if os.path.isfile(macro_markdown) :
    with open(macro_markdown, 'r') as file:
        display(Markdown(file.read()))

## `pyROOT` env check

In [4]:
import ROOT

OFFLINE_MAIN = os.getenv("OFFLINE_MAIN")
if OFFLINE_MAIN is not None:
    display(Markdown(f"via sPHENIX software distribution at `{OFFLINE_MAIN}`"))

via sPHENIX software distribution at `/cvmfs/sphenix.sdcc.bnl.gov/alma9.2-gcc-14.2.0/release/release_new/new.13`

## Plotting source code

In [5]:
import subprocess

try:
    git_url = \
        subprocess.run(['git','remote','get-url','origin'], stdout=subprocess.PIPE)\
        .stdout.decode('utf-8').strip()\
        .replace('git@github.com:','https://github.com/')

    display(Markdown(f"View the source code repository at {git_url}"))
except: # catch *all* exceptions
    #     well do nothing
    pass

View the source code repository at https://github.com/sPHENIX-Collaboration/QA-gallery.git

## JenkinsCI information (if available)

In [6]:
display(Markdown('Some further details about the QA run, if executed under the Jenkins CI:'))

checkrun_repo_commit = os.getenv("checkrun_repo_commit")
if checkrun_repo_commit is not None:
    display(Markdown(f"* The commit being checked is {checkrun_repo_commit}"))
        
ghprbPullLink = os.getenv("ghprbPullLink")
if ghprbPullLink is not None:
    display(Markdown(f"* Link to the pull request: {ghprbPullLink}"))

BUILD_URL =  os.getenv("BUILD_URL")
if BUILD_URL is not None:
    display(Markdown(f"* Link to the build: {BUILD_URL}"))

git_url_macros =  os.getenv("git_url_macros")
sha_macros =  os.getenv("sha_macros")
if git_url_macros is not None:
    display(Markdown(f"* Git repo for macros: {git_url_macros} , which merges `{sha_macros}` and the QA tracking branch"))

RUN_ARTIFACTS_DISPLAY_URL = os.getenv("RUN_ARTIFACTS_DISPLAY_URL")
if RUN_ARTIFACTS_DISPLAY_URL is not None:
    display(Markdown(f"* Download the QA ROOT files: {RUN_ARTIFACTS_DISPLAY_URL}"))

JENKINS_URL  = os.getenv("JENKINS_URL")
if JENKINS_URL is not None:
    display(Markdown(f"Automatically generated by [sPHENIX Jenkins continuous integration]({JENKINS_URL}) [![sPHENIX](https://raw.githubusercontent.com/sPHENIX-Collaboration/utilities/master/jenkins/material/sphenix-logo-white-bg-72p.png)](https://www.sphenix.bnl.gov/web/) &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; [![jenkins.io](https://raw.githubusercontent.com/sPHENIX-Collaboration/utilities/master/jenkins/material/jenkins_logo_title-72p.png)](https://jenkins.io/)"))

Some further details about the QA run, if executed under the Jenkins CI:

* The commit being checked is sPHENIX-Collaboration/coresoftware/8f380afc77e2faed7881c3d40d307203141c8f54

* Link to the build: https://web.sdcc.bnl.gov/jenkins-sphenix/job/sPHENIX/job/test-tracking-reconstruction-prdf/934/

* Git repo for macros: https://github.com/sPHENIX-Collaboration/macros.git , which merges `master` and the QA tracking branch

* Download the QA ROOT files: https://web.sdcc.bnl.gov/jenkins-sphenix/job/sPHENIX/job/test-tracking-reconstruction-prdf/934/display/redirect?page=artifacts

Automatically generated by [sPHENIX Jenkins continuous integration](https://web.sdcc.bnl.gov/jenkins-sphenix/) [![sPHENIX](https://raw.githubusercontent.com/sPHENIX-Collaboration/utilities/master/jenkins/material/sphenix-logo-white-bg-72p.png)](https://www.sphenix.bnl.gov/web/) &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; [![jenkins.io](https://raw.githubusercontent.com/sPHENIX-Collaboration/utilities/master/jenkins/material/jenkins_logo_title-72p.png)](https://jenkins.io/)


# Initialization

In [7]:
%%cpp -d

#include "QA_Draw_Utility.C"

#include <sPhenixStyle.C>

#include <TFile.h>
#include <TLine.h>
#include <TString.h>
#include <TTree.h>
#include <cassert>
#include <cmath>

In [8]:
%%cpp

SetsPhenixStyle();
TVirtualFitter::SetDefaultFitter("Minuit2");

// test sPHENIX lib load
// gSystem->Load("libg4eval.so");

// test libs
// gSystem->ListLibraries();

sPhenixStyle: Applying nominal settings.
sPhenixStyle: ROOT6 mode


In [9]:
%jsroot on

## Inputs and file checks

In [10]:
qa_file_name_new = os.getenv("qa_file_name_new")
if qa_file_name_new is None:
    qa_file_name_new = "prdf_reconstruction53877_qa.root"
    display(Markdown(f"`qa_file_name_new` env not set. use the default `qa_file_name_new={qa_file_name_new}`"))

qa_file_name_ref = os.getenv("qa_file_name_ref")
if qa_file_name_ref is None:
    qa_file_name_ref = "prdf_reconstruction53877_qa.root"
    display(Markdown(f"`qa_file_name_ref` env not set. use the default `qa_file_name_ref={qa_file_name_ref}`"))
elif qa_file_name_ref == 'None':
    qa_file_name_ref = None
    qa_file_name_ref = "prdf_reconstruction53877_qa.root"
    display(Markdown(f"`qa_file_name_ref` = None and we are set to not to use the reference histograms"))
    

display(Markdown(f"Opening QA file at `{qa_file_name_new}`"))
ROOT.gInterpreter.ProcessLine(f"TFile *qa_file_new = new TFile(\"{qa_file_name_new}\");")
ROOT.gInterpreter.ProcessLine(f"const char * qa_file_name_new = \"{qa_file_name_new}\";")

if qa_file_name_ref is not None:
#     qa_file_ref = ROOT.TFile.Open(qa_file_name_ref);

#     assert qa_file_ref.IsOpen()
    display(Markdown(f"Opening QA reference file at `{qa_file_name_ref}`"))
    ROOT.gInterpreter.ProcessLine(f"TFile *qa_file_ref = new TFile(\"{qa_file_name_ref}\");")
    ROOT.gInterpreter.ProcessLine(f"const char *  qa_file_name_ref = \"{qa_file_name_ref}\";")
else:
    ROOT.gInterpreter.ProcessLine(f"TFile *qa_file_ref = nullptr;")
    ROOT.gInterpreter.ProcessLine(f"const char *  qa_file_name_ref = nullptr;")

Opening QA file at `/var/lib/jenkins/workspace/sPHENIX/test-tracking-reconstruction-prdf/macros/TrackingProduction/prdf_reconstruction53877_qa.root`

Opening QA reference file at `reference/prdf_reconstruction53877_qa.root`

#### Eta and phi track distributions

In [11]:
%%cpp




    TCanvas* m_canvas_chip = new TCanvas("m_canvas_chip", "INTT Plots 0", 1000, 800);
    m_canvas_chip->Divide(2, 2);
 
    TH1F *eta_neg_new = (TH1F*)qa_file_new->Get("h_TrackFittingQA_eta_negatively_charged_tracks");
    TH1F *eta_pos_new = (TH1F*)qa_file_new->Get("h_TrackFittingQA_eta_positively_charged_tracks");
    TH1F *intt_state_neg_new = (TH1F*)qa_file_new->Get("h_TrackFittingQA_intt_states_negatively_charged_tracks");
    TH1F *intt_state_pos_new = (TH1F*)qa_file_new->Get("h_TrackFittingQA_intt_states_positively_charged_tracks");
    TH1F *mvtx_state_pos_new = (TH1F*)qa_file_new->Get("h_TrackFittingQA_mvtx_states_positively_charged_tracks");
    TH1F *mvtx_state_neg_new = (TH1F*)qa_file_new->Get("h_TrackFittingQA_mvtx_states_negatively_charged_tracks");
    TH1F *p_pos_new = (TH1F*)qa_file_new->Get("h_TrackFittingQA_p_positively_charged_tracks");
    TH1F *p_neg_new = (TH1F*)qa_file_new->Get("h_TrackFittingQA_p_negatively_charged_tracks");
    TH1F *quality_pos_new = (TH1F*)qa_file_new->Get("h_TrackFittingQA_quality_positively_charged_tracks");
    TH1F *quality_neg_new = (TH1F*)qa_file_new->Get("h_TrackFittingQA_quality_negatively_charged_tracks");
    TH1F *tpc_state_pos_new = (TH1F*)qa_file_new->Get("h_TrackFittingQA_tpc_states_positively_charged_tracks");
    TH1F *tpc_state_neg_new = (TH1F*)qa_file_new->Get("h_TrackFittingQA_tpc_states_negatively_charged_tracks");
    TH1F *tpot_state_pos_new = (TH1F*)qa_file_new->Get("h_TrackFittingQA_tpot_states_positively_charged_tracks");
    TH1F *tpot_state_neg_new = (TH1F*)qa_file_new->Get("h_TrackFittingQA_tpot_states_negatively_charged_tracks");
    TH2F *eta_phi_neg_new = (TH2F*)qa_file_new->Get("h_TrackFittingQA_phi_eta_negatively_charged_tracks");
    TH1 *eta_phi1d_neg_new = eta_phi_neg_new->ProjectionX("eta_phi1d_neg_new");
    TH2F *eta_phi_pos_new = (TH2F*)qa_file_new->Get("h_TrackFittingQA_phi_eta_positively_charged_tracks");
    TH1 *eta_phi1d_pos_new = eta_phi_pos_new->ProjectionX("eta_phi1d_pos_new");
    
    TH1F *eta_neg_ref = (TH1F*)qa_file_ref->Get("h_TrackFittingQA_eta_negatively_charged_tracks");
    TH1F *eta_pos_ref = (TH1F*)qa_file_ref->Get("h_TrackFittingQA_eta_positively_charged_tracks");
    TH1F *intt_state_neg_ref = (TH1F*)qa_file_ref->Get("h_TrackFittingQA_intt_states_negatively_charged_tracks");
    TH1F *intt_state_pos_ref = (TH1F*)qa_file_ref->Get("h_TrackFittingQA_intt_states_positively_charged_tracks");
    TH1F *mvtx_state_pos_ref = (TH1F*)qa_file_ref->Get("h_TrackFittingQA_mvtx_states_positively_charged_tracks");
    TH1F *mvtx_state_neg_ref = (TH1F*)qa_file_ref->Get("h_TrackFittingQA_mvtx_states_negatively_charged_tracks");
    TH1F *p_pos_ref = (TH1F*)qa_file_ref->Get("h_TrackFittingQA_p_positively_charged_tracks");
    TH1F *p_neg_ref = (TH1F*)qa_file_ref->Get("h_TrackFittingQA_p_negatively_charged_tracks");
    TH1F *quality_pos_ref = (TH1F*)qa_file_ref->Get("h_TrackFittingQA_quality_positively_charged_tracks");
    TH1F *quality_neg_ref = (TH1F*)qa_file_ref->Get("h_TrackFittingQA_quality_negatively_charged_tracks");
    TH1F *tpc_state_pos_ref = (TH1F*)qa_file_ref->Get("h_TrackFittingQA_tpc_states_positively_charged_tracks");
    TH1F *tpc_state_neg_ref = (TH1F*)qa_file_ref->Get("h_TrackFittingQA_tpc_states_negatively_charged_tracks");
    TH1F *tpot_state_pos_ref = (TH1F*)qa_file_ref->Get("h_TrackFittingQA_tpot_states_positively_charged_tracks");
    TH1F *tpot_state_neg_ref = (TH1F*)qa_file_ref->Get("h_TrackFittingQA_tpot_states_negatively_charged_tracks");
     TH2F *eta_phi_neg_ref = (TH2F*)qa_file_ref->Get("h_TrackFittingQA_phi_eta_negatively_charged_tracks");
    TH1 *eta_phi1d_neg_ref = eta_phi_neg_ref->ProjectionX("eta_phi1d_neg_ref");
    TH2F *eta_phi_pos_ref = (TH2F*)qa_file_ref->Get("h_TrackFittingQA_phi_eta_positively_charged_tracks");
    TH1 *eta_phi1d_pos_ref= eta_phi_pos_ref->ProjectionX("eta_phi1d_pos_ref");
 
   m_canvas_chip->cd(1);
   DrawReference(eta_neg_ref, eta_neg_new);
    m_canvas_chip->cd(2);
    DrawReference(eta_pos_new, eta_pos_ref);
    
    m_canvas_chip->cd(3);
    DrawReference(eta_phi1d_neg_new, eta_phi1d_neg_ref);
    m_canvas_chip->cd(4);
    DrawReference(eta_phi1d_pos_new, eta_phi1d_pos_ref);
    
   
  
    m_canvas_chip->Update();
 
    std::cout << "DrawChipInfo completed" << std::endl;


DrawChipInfo completed


#### Silicon states

In [12]:
%%cpp

    TCanvas *can1 = new TCanvas("can1","can1",1000,800);
    
    can1->Divide(2,2);
    
    can1->cd(1);
    DrawReference(mvtx_state_neg_new, mvtx_state_neg_ref);
    
    can1->cd(2);
    DrawReference(mvtx_state_pos_new, mvtx_state_pos_ref);
    
    can1->cd(3);
    DrawReference(intt_state_neg_new, intt_state_neg_ref);
    
    can1->cd(4);
    DrawReference(intt_state_pos_new, intt_state_pos_ref);
    can1->Update();

#### Tpc and TPOT states

In [13]:
%%cpp

TCanvas *can2 = new TCanvas("can2","can2",1000,800);
can2->Divide(2,2);

can2->cd(1);
DrawReference(tpc_state_neg_new, tpc_state_neg_ref);
can2->cd(2);
DrawReference(tpc_state_pos_new, tpc_state_pos_ref);
can2->cd(3);
DrawReference(tpot_state_neg_new, tpot_state_neg_ref);
can2->cd(4);
DrawReference(tpot_state_pos_new, tpot_state_pos_ref);
can2->Update();


#### Momentum and quality

In [14]:
%%cpp

TCanvas *can3 = new TCanvas("can3","can3",1000,800);
can3->Divide(2,2);
can3->cd(1);
DrawReference(p_neg_new, p_neg_ref);
can3->cd(2);
DrawReference(p_pos_new, p_pos_ref);
can3->cd(3);
DrawReference(quality_neg_new, quality_neg_ref);
can3->cd(4);
DrawReference(quality_pos_new, quality_pos_ref);
can3->Update();

In [15]:
%%cpp

KSTestSummary::getInstance()->make_summary_txt("QA-trackfit.txt");
KSTestSummary::getInstance()->make_summary_TCanvas()->Draw();

This notebook contains 16 KSTets: combined Chi2/nDoF = -0 / 32, and combined __p-Value = 1__
